## Unión y depuración de la data

En este notebook uniremos toda la data recopilada anteriormente y crearemos un único DataFrame.  
Posteriormente realizaremos una limpieza de los datos.

In [7]:
import glob
import pandas as pd
from functools import reduce
import numpy as np
pd.options.mode.chained_assignment = None #evitar la advertencia: SettingWithCopyWarning

En lo posterior, necesito que el notebook muestre todas las columnas del DataFrame:

In [8]:
pd.set_option('display.max_columns', None)

Unimos todos los csv para crear un único DataFrame:

In [9]:
dfLista = []
ruta = '/home/dsc/Python/Despoblacion/Data' 
csv_files = glob.glob(ruta + "/*11.csv")
csv_files.sort()

In [10]:
for file in csv_files:
    df = pd.read_csv(file, dtype = {'Codigo Provincia' : str, 'Codigo Municipio' : str})
    dfLista.append(df)

In [11]:
Despoblacion = reduce(lambda  left,right: pd.merge(left,right,on=['Codigo Municipio'],
                                            how='left'), dfLista)
Despoblacion.head()

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas
0,Araba/Álava,01,Alegría-Dulantzi,01001,2803.0,1466.0,1337.0,19.95,140.50,568.0,37.0,18.0,14.0,784.0,617,453,164,1992,935,1057,NaN,NaN,NaN,NaN,1.0,4.0,25.0,1.0,1.0,150.0,127.0,277.0,279.0,268.0,547.0,667.0,1346.0,1249.0,645,1965,240,1250,1410,30,90,75,325,1835,255,120,120,230,390,60,60,335,195,2635,195,685,705,1125,105,40,NaN,NaN,NaN,NaN,NaN,655,425,1947.0,1664.0,1843.0,828.73267,1124.34,35.0,6.0
1,Araba/Álava,01,Amurrio,01002,10014.0,5030.0,4984.0,96.30,103.99,219.0,104.0,37.0,88.0,4397.0,5861,4843,1018,8393,3742,4651,NaN,NaN,NaN,NaN,2.0,9.0,185.0,3.0,NaN,409.0,334.0,742.0,869.0,918.0,1787.0,1316.0,5154.0,4435.0,1285,6935,1795,3975,5120,120,130,675,1750,6975,1955,245,320,745,1285,500,330,955,730,9265,730,2545,2945,2745,690,360,NaN,NaN,NaN,NaN,NaN,2360,1505,6906.0,6196.0,2166.0,3579.79700,5568.50,381.0,367.0
2,Araba/Álava,01,Añana,01049,168.0,86.0,82.0,21.92,7.66,574.0,1.0,0.0,7.0,30.0,20,14,6,50,16,34,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,1.0,6.0,5.0,11.0,12.0,10.0,22.0,181.0,214.0,209.0,15,105,40,65,75,*,*,15,35,102,50,5,10,10,20,10,15,5,30,105,30,45,25,60,5,*,NaN,NaN,NaN,NaN,NaN,15,60,130.0,65.0,229.0,90.70000,1212.00,20.0,9.0
3,Araba/Álava,01,Aramaio,01003,1496.0,784.0,712.0,73.02,20.49,333.0,13.0,3.0,10.0,224.0,141,88,53,322,177,145,NaN,NaN,NaN,NaN,1.0,2.0,8.0,NaN,NaN,23.0,34.0,57.0,42.0,38.0,80.0,423.0,706.0,681.0,265,955,290,725,680,*,*,95,250,980,325,25,70,45,205,45,70,60,125,1385,125,380,270,545,120,65,NaN,NaN,NaN,NaN,NaN,315,260,1236.0,1014.0,431.0,616.46180,2055.80,191.0,215.0
4,Araba/Álava,01,Armiñón,01006,229.0,112.0,117.0,12.97,17.66,467.0,2.0,0.0,3.0,137.0,125,90,35,139,60,79,NaN,NaN,NaN,NaN,NaN,2.0,7.0,NaN,NaN,4.0,4.0,8.0,24.0,38.0,62.0,141.0,153.0,153.0,40,150,35,100,110,*,*,10,30,160,35,10,15,30,20,5,10,*,20,200,20,45,95,60,5,*,NaN,NaN,NaN,NaN,NaN,65,30,205.0,145.0,0.0,NaN,859.32,19.0,2.0


In [42]:
Despoblacion.shape

(8116, 78)

In [37]:
Formato = Despoblacion.dtypes
print(Formato.to_string())

Nombre Provincia                            object
Codigo Provincia                            object
Nombre Municipio                            object
Codigo Municipio                            object
Poblacion Total                            float64
Poblacion Hombre                           float64
Poblacion Mujer                            float64
Superficie Km2                             float64
Densidad Poblacion                         float64
Altitud Municipios                         float64
Nacimientos                                float64
Matrimonios                                float64
Defunciones                                float64
Total Regimen SS                           float64
Total Contratos                              int64
Contratos Hombres                            int64
Contratos Mujeres                            int64
Total Paro Registrado                        int64
Parados Hombres                              int64
Parados Mujeres                

Transformamos todos los valores del DataFrame en numéricos con dos decimales.  
Antes de ello, en la fase previa de recopilación de la data, se detectó que había tablas con variables que toman el valor \*.
Estos valores corresponden a variables extraídas del INE donde, por secreto estadístico, los registros que tienen un valor inferior a 5, les asigunan un \*.   
Localizamos las columnas que contienen \* y lo sustituimos por la media:

In [38]:
ColumnasAsterisco = Despoblacion.columns[Despoblacion.isin(['*']).any()]
ColumnasAsterisco

Index(['Edad < 16', '16 =< Edad =<64', 'Edad >= 65', 'Soltero/a', 'Casado/a',
       'Separado/a', 'Divorciado/a', 'Viudo/a', 'Nº de Jubilados',
       'Empresario que emplea personal', 'Empresario que no emplea personal',
       '1 hijo/a', '2 hijos/as', '3 hijos/as', '4 o más hijos/as',
       'Nº de Alquileres', 'Hogares Unipersonales', 'Hogares Familiares',
       '1 persona', '2 personas', '3 personas', '4 personas', '5 personas',
       '6 o más personas', 'Nº Hogares si Internet', 'Nº Hogares no Internet'],
      dtype='object')

In [39]:
Despoblacion[ColumnasAsterisco] = Despoblacion[ColumnasAsterisco].replace('*', 2)

Ahora si que ya podemos realizar el cambio de formato desde la columna número 4 en adelante:

In [40]:
Columnas = Despoblacion.columns[4 : ]
Despoblacion[Columnas] = round(Despoblacion[Columnas].apply(pd.to_numeric), 2)

In [41]:
Formato = Despoblacion.dtypes
print(Formato.to_string())

Nombre Provincia                            object
Codigo Provincia                            object
Nombre Municipio                            object
Codigo Municipio                            object
Poblacion Total                            float64
Poblacion Hombre                           float64
Poblacion Mujer                            float64
Superficie Km2                             float64
Densidad Poblacion                         float64
Altitud Municipios                         float64
Nacimientos                                float64
Matrimonios                                float64
Defunciones                                float64
Total Regimen SS                           float64
Total Contratos                              int64
Contratos Hombres                            int64
Contratos Mujeres                            int64
Total Paro Registrado                        int64
Parados Hombres                              int64
Parados Mujeres                

La dimensión de nuestro DataFrame es de 8.116 filas, coincidiendo con el número de municipios en España en 2011
No obstante, veamos si hay algún Codigo Municipio duplicado:

In [45]:
Despoblacion[Despoblacion['Codigo Municipio'].duplicated()]

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Total Regimen SS,Total Contratos,Contratos Hombres,Contratos Mujeres,Total Paro Registrado,Parados Hombres,Parados Mujeres,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Con Estudios,Nº de Jubilados,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet,Nº Vehiculos,Nº Conductores,Deuda (Miles de €),Total impuestos (Miles de €),Superficie (Ha) Agricolas,Nº Explotaciones Agrícolas,Nº Explotaciones Ganaderas


Eliminamos aquellos municipios que tiene una densidad menor a 12,5, municipios ya despoblados en 2011 dificilmente pueden volver a áreas pobladas:

In [15]:
Despob1acion = Despoblacion.loc[Despoblacion['Densidad Poblacion'] >= 12.5 ]

In [16]:
Despob1acion.head()

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Regimen General,Regimen Agrario,Regimen Mar,Regimen Hogar,Regimen Autonomos,Regimen Carbon,Total Regimen SS,Total Contratos,Contratos iniciales indefinidos hombres,Contratos iniciales temporales hombres,Contratos convertidos en indefinidos hombres,Contratos iniciales indefinidos mujeres,Contratos iniciales temporales mujeres,Contratos convertidos en indefinidos mujeres,Contratos Agricultura,Contratos Industria,Contratos Construcción,Contratos Servicios,Total Paro Registrado,Paro hombre edad < 25,Paro hombre edad 25 -45,Paro hombre edad >=45,Paro mujer edad < 25,Paro mujer edad 25 -45,Paro mujer edad >=45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Estudios Primarios o Secundarios,Estudios Superiones,Nº de Jubilados,Nº de Estudiantes,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet
0,Araba/Álava,01,Alegría-Dulantzi,01001,2803.0,1466.0,1337.0,19.95,140.50,568.0,37.0,18.0,14.0,576.0,5.0,0.0,10.0,193.0,0.0,784.0,31,0,24,0,0,7,0,0,3,13,15,1992,86,552,297,97,737,223,35,444,149,1237,127,0.0,17.0,12.0,62.0,1.0,4.0,25.0,1.0,1.0,150.0,127.0,277.0,279.0,268.0,547.0,667.0,1346.0,1249.0,645,1965,240,1250,1410,30,90,75,325,1380,455,255,650,120,120,230,390,60,60,335,195,2635,195,685,705,1125,105,40,NaN,NaN,NaN,NaN,NaN,655,425
1,Araba/Álava,01,Amurrio,01002,10014.0,5030.0,4984.0,96.30,103.99,219.0,104.0,37.0,88.0,3713.0,15.0,0.0,47.0,622.0,0.0,4397.0,382,3,276,15,1,82,5,2,227,14,139,8393,535,1816,1391,337,2243,2071,130,1683,941,5036,603,8.0,61.0,33.0,261.0,2.0,9.0,185.0,3.0,NaN,409.0,334.0,742.0,869.0,918.0,1787.0,1316.0,5154.0,4435.0,1285,6935,1795,3975,5120,120,130,675,1750,5430,1545,1955,1655,245,320,745,1285,500,330,955,730,9265,730,2545,2945,2745,690,360,NaN,NaN,NaN,NaN,NaN,2360,1505
3,Araba/Álava,01,Aramaio,01003,1496.0,784.0,712.0,73.02,20.49,333.0,13.0,3.0,10.0,148.0,2.0,0.0,5.0,69.0,0.0,224.0,6,0,4,0,0,2,0,0,0,0,6,322,2,69,106,5,79,61,0,146,3,160,13,3.0,4.0,3.0,12.0,1.0,2.0,8.0,NaN,NaN,23.0,34.0,57.0,42.0,38.0,80.0,423.0,706.0,681.0,265,955,290,725,680,2,2,95,250,705,275,325,315,25,70,45,205,45,70,60,125,1385,125,380,270,545,120,65,NaN,NaN,NaN,NaN,NaN,315,260
4,Araba/Álava,01,Armiñón,01006,229.0,112.0,117.0,12.97,17.66,467.0,2.0,0.0,3.0,115.0,0.0,0.0,0.0,22.0,0.0,137.0,3,0,0,0,0,1,2,0,3,0,0,139,5,45,10,3,44,32,0,39,21,79,0,0.0,3.0,0.0,4.0,NaN,2.0,7.0,NaN,NaN,4.0,4.0,8.0,24.0,38.0,62.0,141.0,153.0,153.0,40,150,35,100,110,2,2,10,30,140,20,35,40,10,15,30,20,5,10,2,20,200,20,45,95,60,5,2,NaN,NaN,NaN,NaN,NaN,65,30
6,Araba/Álava,01,Arrazua-Ubarrundia,01008,951.0,508.0,443.0,57.56,16.52,528.0,11.0,1.0,7.0,148.0,2.0,0.0,6.0,127.0,0.0,283.0,11,0,7,0,0,4,0,0,0,2,9,526,24,95,104,32,137,134,10,106,76,316,18,6.0,6.0,4.0,26.0,1.0,2.0,4.0,1.0,NaN,16.0,19.0,35.0,75.0,88.0,163.0,446.0,484.0,483.0,140,675,135,435,430,2,25,60,165,455,180,175,160,80,50,60,120,50,25,35,85,840,85,160,290,290,90,35,NaN,NaN,NaN,NaN,NaN,180,175


In [17]:
Despob1acion.shape

(4411, 96)

Analizamos los valores nulos:

In [18]:
Valores = Despob1acion.isnull().sum()
print(Valores.to_string())

Nombre Provincia                                   0
Codigo Provincia                                   0
Nombre Municipio                                   0
Codigo Municipio                                   0
Poblacion Total                                    0
Poblacion Hombre                                   0
Poblacion Mujer                                    0
Superficie Km2                                     0
Densidad Poblacion                                 0
Altitud Municipios                                 1
Nacimientos                                        0
Matrimonios                                        0
Defunciones                                        0
Regimen General                                   17
Regimen Agrario                                   17
Regimen Mar                                       17
Regimen Hogar                                     17
Regimen Autonomos                                 17
Regimen Carbon                                

**Altitud Municipios**: solo un valor nulo.  
Se busca el valor y se completa:

In [19]:
Despob1acion[Despob1acion['Altitud Municipios'].isnull()]

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Regimen General,Regimen Agrario,Regimen Mar,Regimen Hogar,Regimen Autonomos,Regimen Carbon,Total Regimen SS,Total Contratos,Contratos iniciales indefinidos hombres,Contratos iniciales temporales hombres,Contratos convertidos en indefinidos hombres,Contratos iniciales indefinidos mujeres,Contratos iniciales temporales mujeres,Contratos convertidos en indefinidos mujeres,Contratos Agricultura,Contratos Industria,Contratos Construcción,Contratos Servicios,Total Paro Registrado,Paro hombre edad < 25,Paro hombre edad 25 -45,Paro hombre edad >=45,Paro mujer edad < 25,Paro mujer edad 25 -45,Paro mujer edad >=45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Estudios Primarios o Secundarios,Estudios Superiones,Nº de Jubilados,Nº de Estudiantes,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet
2183,"Coruña, A",15,Oza dos Ríos,15063,3229.0,1622.0,1607.0,72.14,44.76,NaN,23.0,11.0,45.0,333.0,2.0,2.0,14.0,305.0,0.0,656.0,15,1,9,2,0,2,1,0,0,8,7,2512,183,585,427,42,733,542,41,376,422,1361,312,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,66.0,61.0,127.0,65.0,52.0,117.0,1857.0,2084.0,2043.0,335,1855,1030,1175,1585,2,80,335,1130,1430,220,1180,300,195,240,305,470,170,70,2,285,2785,285,915,1125,585,125,185,NaN,NaN,NaN,NaN,NaN,525,795


In [20]:
Despob1acion.loc[2183, 'Altitud Municipios'] = 231

**Afiliados SS**: Los campos Regimen General, Regimen Agrario, Regimen Mar, Regimen Hogar, Regimen Autonomos , Regimen Carbon, Total Regimen SS tienen 17 valores nulos.  
La tabla de afiliados no contiene los 17 municipios correspondientes:

In [21]:
Despob1acion[Despob1acion['Regimen General'].isnull()].head(1)

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Regimen General,Regimen Agrario,Regimen Mar,Regimen Hogar,Regimen Autonomos,Regimen Carbon,Total Regimen SS,Total Contratos,Contratos iniciales indefinidos hombres,Contratos iniciales temporales hombres,Contratos convertidos en indefinidos hombres,Contratos iniciales indefinidos mujeres,Contratos iniciales temporales mujeres,Contratos convertidos en indefinidos mujeres,Contratos Agricultura,Contratos Industria,Contratos Construcción,Contratos Servicios,Total Paro Registrado,Paro hombre edad < 25,Paro hombre edad 25 -45,Paro hombre edad >=45,Paro mujer edad < 25,Paro mujer edad 25 -45,Paro mujer edad >=45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Estudios Primarios o Secundarios,Estudios Superiones,Nº de Jubilados,Nº de Estudiantes,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet
113,Albacete,02,Pozo Cañada,02901,2895.0,1462.0,1433.0,116.83,24.78,830.0,25.0,12.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,2,18,0,0,9,0,12,1,2,14,4180,342,816,825,255,1323,619,285,310,917,2297,371,NaN,NaN,NaN,NaN,1.0,NaN,14.0,NaN,NaN,82.0,58.0,140.0,62.0,65.0,127.0,995.0,1355.0,1308.0,430,1915,520,1105,1560,2,2,140,930,1310,165,510,480,105,60,105,365,180,160,15,140,2700,140,715,555,1250,195,15,38.0,392.0,1341.0,38.0,16.0,365,670


In [22]:
Afiliaciones11 = pd.read_csv('/home/dsc/Python/Despoblacion/Data/06_Output_Afiliados_SS_11.csv' ,dtype='object')

In [23]:
Afiliaciones11[Afiliaciones11['Codigo Municipio'] == '02901']

,Codigo Municipio,Regimen General,Regimen Agrario,Regimen Mar,Regimen Hogar,Regimen Autonomos,Regimen Carbon,Total Regimen SS


Eliminamos los 17 registros:

In [24]:
Columnas = Afiliaciones11.columns[1:]
Columnas.tolist()

['Regimen General',
 'Regimen Agrario',
 'Regimen Mar',
 'Regimen Hogar',
 'Regimen Autonomos',
 'Regimen Carbon',
 'Total Regimen SS']

In [25]:
Despob1acion = Despob1acion.dropna(subset = Columnas)

**Número de transacciones anuales 2011**: Casualmente son 4 municipios que posteriormente, en 2016,se fusionaron: Cercedo-Cotobade, Oza-Cesuras. Vemos si en el DataFrame de transacciones aparecen los municipios fusionados:


In [26]:
Despob1acion[Despob1acion['Número de transacciones anuales 2011'].isnull()].head()

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Regimen General,Regimen Agrario,Regimen Mar,Regimen Hogar,Regimen Autonomos,Regimen Carbon,Total Regimen SS,Total Contratos,Contratos iniciales indefinidos hombres,Contratos iniciales temporales hombres,Contratos convertidos en indefinidos hombres,Contratos iniciales indefinidos mujeres,Contratos iniciales temporales mujeres,Contratos convertidos en indefinidos mujeres,Contratos Agricultura,Contratos Industria,Contratos Construcción,Contratos Servicios,Total Paro Registrado,Paro hombre edad < 25,Paro hombre edad 25 -45,Paro hombre edad >=45,Paro mujer edad < 25,Paro mujer edad 25 -45,Paro mujer edad >=45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Estudios Primarios o Secundarios,Estudios Superiones,Nº de Jubilados,Nº de Estudiantes,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet
2145,"Coruña, A",15,Cesuras,15026,2225.0,1090.0,1135.0,79.46,28.00,231.0,3.0,9.0,35.0,67.0,6.0,0.0,11.0,241.0,0.0,325.0,3,1,0,1,0,1,0,0,0,0,3,1396,76,350,514,45,243,168,33,208,427,606,122,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,21.0,22.0,43.0,25.0,23.0,48.0,1410.0,1417.0,1402.0,2,1340,495,870,995,2,2,310,225,1025,2,520,2,2,2,2,570,2,2,2,2,2090,2,105,455,145,1455,2,NaN,NaN,NaN,NaN,NaN,2,355
2183,"Coruña, A",15,Oza dos Ríos,15063,3229.0,1622.0,1607.0,72.14,44.76,231.0,23.0,11.0,45.0,333.0,2.0,2.0,14.0,305.0,0.0,656.0,15,1,9,2,0,2,1,0,0,8,7,2512,183,585,427,42,733,542,41,376,422,1361,312,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,66.0,61.0,127.0,65.0,52.0,117.0,1857.0,2084.0,2043.0,335,1855,1030,1175,1585,2,80,335,1130,1430,220,1180,300,195,240,305,470,170,70,2,285,2785,285,915,1125,585,125,185,NaN,NaN,NaN,NaN,NaN,525,795
5281,Pontevedra,36,Cerdedo,36011,2297.0,1063.0,1234.0,134.70,17.05,374.0,7.0,3.0,32.0,119.0,2.0,0.0,6.0,107.0,0.0,234.0,45,0,21,0,2,17,5,0,10,7,28,1319,62,309,299,28,359,262,62,235,357,535,130,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,75.0,55.0,130.0,36.0,28.0,64.0,1108.0,1192.0,1189.0,130,1010,710,540,890,2,90,315,715,795,160,750,100,25,60,250,205,140,75,85,305,1540,305,460,655,345,85,2,NaN,NaN,NaN,NaN,NaN,230,630
5282,Pontevedra,36,Cotobade,36012,4432.0,2101.0,2331.0,79.84,55.51,374.0,26.0,12.0,71.0,295.0,9.0,0.0,27.0,274.0,0.0,605.0,12,1,4,1,0,5,1,0,3,1,8,4957,216,1347,1065,254,1231,844,190,699,1326,2243,499,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,91.0,107.0,198.0,95.0,70.0,165.0,2544.0,2565.0,2561.0,525,2580,1295,1720,2245,2,2,390,1840,1770,240,1320,540,2,270,195,435,205,245,295,340,3880,340,900,915,595,445,1200,NaN,NaN,NaN,NaN,NaN,540,995


In [27]:
Transacciones11 = pd.read_csv('/home/dsc/Python/Despoblacion/Data/12_Output_Transacciones_Inmobiliarias_11.csv' ,dtype='object')

In [28]:
Transacciones11[Transacciones11['Codigo Municipio'] == '36902']#Cercedo-Cotobade

,Codigo Municipio,Número de transacciones anuales 2011
5344,36902,NaN


In [29]:
Transacciones11[Transacciones11['Codigo Municipio'] == '15902'] #Oza-Cesuras

,Codigo Municipio,Número de transacciones anuales 2011
2218,15902,27.0


Eliminamos los registros de Cercedo y Cotobade.   
Tenemos el dato global de Oza-Cesuras, estimamos: la población es un 60/40, asignamos 16/11:

In [30]:
Despob1acion.loc[2145, 'Número de transacciones anuales 2011'] = 11

In [31]:
Despob1acion.loc[2183, 'Número de transacciones anuales 2011'] = 16

In [32]:
Despob1acion = Despob1acion.dropna(subset = ['Número de transacciones anuales 2011'])

**Nº centros sanitarios, educativos, residencias mayores, instituciones religiosas**: Los valores nulos corresponden a la falta de dichos centros en esos municipios.   
Los susbstituimos por ceros:


In [33]:
ColumnasNan = ['Nº Centros Sanitarios', 'Nº Centros Educativos', 
               'Nº Residencias Mayores', 'Nº Instituciones Religiosas y Militares']
Despob1acion[ColumnasNan] = Despob1acion[ColumnasNan].replace(np.nan, 0)

**Nº de extranjeros por municipio**: hay 51 nulos en hombres, 35 en mujeres y 13 en el total.
Además, si nos fijamos en la población total de estos municipios, está muy por debajo de la media.  
Entendemos los valores nulos como cero casos muestrales.

In [34]:
round(Despob1acion['Poblacion Total'].mean(), 2)

10435.39

In [35]:
NulosExtranjeros = Despob1acion[Despob1acion['Nº Total Extranjeros'].isnull()]
(NulosExtranjeros['Poblacion Total'] < 10435.39).sum()

13

In [36]:
NulosExtranjeros['Poblacion Total']

384     145.0
699     298.0
1108    182.0
1567    458.0
1619    482.0
2439    433.0
3626    216.0
4803     55.0
5451    114.0
5539    136.0
7621    227.0
7639    165.0
8028     53.0
Name: Poblacion Total, dtype: float64

In [37]:
ColumnasNan = ['Nº Hombres Extranjeros', 'Nº Mujeres Extranjeras', 'Nº Total Extranjeros']
Despob1acion[ColumnasNan] = Despob1acion[ColumnasNan].replace(np.nan, 0)

**Nº de personas otras comunidades autónomas**: Nº Hombres Otra CCAA hay 8 nulos, Nº Mujeres Otra CCAA hay 4 y en el total no hay nulos.  
Los nulos de Hombres y Mujeres son ceros.

In [38]:
ColumnasNan = ['Nº Hombres Otra CCAA', 'Nº Mujeres Otra CCAA']
Despob1acion[ColumnasNan] = Despob1acion[ColumnasNan].replace(np.nan, 0)

**Edad Media Edificios y número de establecimientos**: Tienen el mismo número de nulos, 409.   
En este caso tenemos muchos registros nulos de los que no disponemos de sus valores.  
Los eliminamos:

In [39]:
Columnas = ['Edad Media Edificios', 'Uso Industrial', 'Uso Residelcial', 'Uso Terciario', 'Otros Usos']
Despob1acion = Despob1acion.dropna(subset = Columnas)

Al eliminar los nulos de edad media de edificios y número de establecimientos, también elimnamos los últimos 4 valores nulos que quedan en el DataFrame correspondiente a las columnas de nº de establecimientos por tipología.  

In [40]:
(Despob1acion.isnull().sum()).sum()

0

In [41]:
Despob1acion.head()

,Nombre Provincia,Codigo Provincia,Nombre Municipio,Codigo Municipio,Poblacion Total,Poblacion Hombre,Poblacion Mujer,Superficie Km2,Densidad Poblacion,Altitud Municipios,Nacimientos,Matrimonios,Defunciones,Regimen General,Regimen Agrario,Regimen Mar,Regimen Hogar,Regimen Autonomos,Regimen Carbon,Total Regimen SS,Total Contratos,Contratos iniciales indefinidos hombres,Contratos iniciales temporales hombres,Contratos convertidos en indefinidos hombres,Contratos iniciales indefinidos mujeres,Contratos iniciales temporales mujeres,Contratos convertidos en indefinidos mujeres,Contratos Agricultura,Contratos Industria,Contratos Construcción,Contratos Servicios,Total Paro Registrado,Paro hombre edad < 25,Paro hombre edad 25 -45,Paro hombre edad >=45,Paro mujer edad < 25,Paro mujer edad 25 -45,Paro mujer edad >=45,Paro Agricultura,Paro Industria,Paro Construcción,Paro Servicios,Paro Sin empleo Anterior,Nº Establecimientos Agricultura,Nº Establecimientos Industria,Nº Establecimientos Construccion,Nº Establecimientos Servicios,Nº Centros Sanitarios,Nº Centros Educativos,Número de transacciones anuales 2011,Nº Residencias Mayores,Nº Instituciones Religiosas y Militares,Nº Hombres Extranjeros,Nº Mujeres Extranjeras,Nº Total Extranjeros,Nº Hombres Otra CCAA,Nº Mujeres Otra CCAA,Nº Total Otra CCAA,Nº edificios,Nº Inmuebles,Nº viviendas,Edad < 16,16 =< Edad =<64,Edad >= 65,Soltero/a,Casado/a,Separado/a,Divorciado/a,Viudo/a,Sin Estudios,Estudios Primarios o Secundarios,Estudios Superiones,Nº de Jubilados,Nº de Estudiantes,Empresario que emplea personal,Empresario que no emplea personal,1 hijo/a,2 hijos/as,3 hijos/as,4 o más hijos/as,Nº de Alquileres,Hogares Unipersonales,Hogares Familiares,1 persona,2 personas,3 personas,4 personas,5 personas,6 o más personas,Edad Media Edificios,Uso Industrial,Uso Residelcial,Uso Terciario,Otros Usos,Nº Hogares si Internet,Nº Hogares no Internet
51,Albacete,02,Abengibre,02001,902.0,457.0,445.0,30.77,29.31,634.0,5.0,3.0,19.0,149.0,23.0,0.0,2.0,92.0,0.0,266.0,9,0,7,0,0,2,0,1,1,0,7,729,52,143,121,31,225,157,154,116,23,382,54,3.0,5.0,1.0,13.0,1.0,2.0,3.0,0.0,0.0,40.0,28.0,68.0,16.0,28.0,44.0,455.0,462.0,462.0,75,545,240,295,485,2,2,60,370,320,70,250,125,30,115,30,145,50,35,25,50,795,50,185,205,245,100,75,42.0,183.0,461.0,11.0,4.0,100,200
53,Albacete,02,Albacete,02003,171390.0,84288.0,87102.0,1126.99,152.08,681.0,1957.0,658.0,1156.0,55548.0,2403.0,0.0,924.0,10621.0,0.0,69496.0,4525,108,2061,104,73,2092,87,472,168,290,3595,206017,15279,53061,29416,13377,59488,35396,8065,20335,28615,128710,20292,270.0,578.0,667.0,5639.0,21.0,128.0,1099.0,9.0,1.0,7104.0,6570.0,13674.0,8684.0,9293.0,17977.0,15346.0,89834.0,83229.0,29940,117445,23645,79255,76720,2205,4325,8525,32750,75040,33305,26480,36400,5255,6020,9455,21910,8840,6490,20340,13445,157585,13445,33800,39760,58910,16345,8765,26.0,40004.0,82879.0,7400.0,141.0,37410,25570
54,Albacete,02,Albatana,02004,794.0,403.0,391.0,30.50,26.03,582.0,5.0,3.0,5.0,33.0,39.0,0.0,2.0,69.0,0.0,143.0,12,0,9,0,0,2,1,6,0,0,6,898,59,118,159,44,192,326,82,99,124,555,38,1.0,4.0,1.0,9.0,1.0,2.0,2.0,0.0,0.0,22.0,19.0,41.0,38.0,40.0,78.0,536.0,570.0,570.0,90,470,230,285,415,2,2,60,365,275,35,235,135,25,45,2,120,65,35,2,60,700,60,205,140,220,105,55,52.0,80.0,454.0,8.0,3.0,35,260
59,Albacete,02,Almansa,02009,25432.0,12631.0,12801.0,531.82,47.82,687.0,254.0,101.0,214.0,5480.0,159.0,0.0,58.0,1649.0,0.0,7346.0,308,8,139,20,9,115,17,16,85,23,184,38565,2335,8082,5791,2262,9743,10352,1038,10969,3215,19231,4112,44.0,228.0,82.0,633.0,2.0,22.0,143.0,2.0,0.0,789.0,777.0,1567.0,1738.0,1749.0,3487.0,3823.0,12074.0,11752.0,4120,17035,4085,10740,11930,280,705,1580,6890,11545,2620,4695,4955,960,1020,1580,3525,1245,885,1195,2145,22890,2145,5270,6415,9580,1190,640,29.0,5131.0,12230.0,587.0,33.0,5250,4405
60,Albacete,02,Alpera,02010,2438.0,1261.0,1177.0,178.47,13.66,852.0,15.0,7.0,26.0,210.0,96.0,0.0,5.0,207.0,0.0,518.0,27,0,18,0,0,9,0,1,4,10,12,3354,234,541,615,263,906,795,256,721,382,1652,343,20.0,15.0,8.0,38.0,1.0,

In [42]:
Despob1acion.to_csv('Despoblacion_11.csv', header = True, index = False)